In [ ]:
#For Kaggle
#date=''

#For WL data
date='20241118'
source='WL'

saveImg=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import countrymanagement as cm
import mccmanagement
reload(fd)
reload(sd)
reload(di)
reload(cm)
reload(mccmanagement)

print('done')

In [ ]:
import dataimport as di
import pandas as pd 
   
dfTrx = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Remove unless columns

In [ ]:
# remove db_uuid issuer_id cluster_profile

dfTrx= dfTrx.drop(columns=['issuer_id', 'cluster_profile'])
dfTrx.head()

# Fill missing values

In [ ]:
import numpy as np

nan_count = dfTrx.isnull().sum()
print('Number of NaN values:', nan_count)


dfTrx['mcd_fraud_score'].fillna(np.mean(dfTrx['mcd_fraud_score']), inplace=True)
dfTrx['vaa_score'].fillna(np.mean(dfTrx['vaa_score']), inplace=True)
dfTrx.head()

# Grouping countries

In [ ]:
dfTrx['country_group'] = dfTrx['term_country'].apply(cm.get_country_group)
#dfTrx= dfTrx.drop(columns=['term_country'])
dfTrx.head()

# Grouping MCC

In [ ]:
dfTrx['mcc_group'] = dfTrx['term_mcc'].apply(mccmanagement.get_mcc_group_citybank)
dfTrx['mcc_group'] = np.where(dfTrx.term_mcc.isin([mccmanagement.get_mcc_group_ATM]),'ATM',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['AGRICULTURAL']), 'OTHER',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['CONTRACTED_SERVICES']), 'OTHER',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['AIRLINES']),'OTHER',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['CAR_RENTAL']), 'OTHER',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['LODGING']),'OTHER',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['TRANSPORTATION_SERVICES']), 'OTHER',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['MISCELLANOUS_STORES']), 'OTHER',dfTrx['mcc_group'] )
dfTrx['mcc_group'] = np.where(dfTrx.mcc_group.isin(['BUSINESS_SERVICES']), 'OTHER',dfTrx['mcc_group'] )
dfTrx['term_mcc'] = dfTrx['term_mcc'].astype('str')
dfTrx.head()

# Fix reversal

In [ ]:
dfTrx = di.reversal_fix(dfTrx)
dfTrx.head()

# Amount log10 transformation 

In [ ]:
import numpy as np

dfTrx['trx_amount_log10']=np.log10(1+dfTrx['trx_amount'])
dfTrx= dfTrx.drop(columns=['trx_amount'])
dfTrx.head()

# Get card profile

In [ ]:
import pandas as pd

dfCardProfile = pd.read_csv('../data/processed/'+'WLCardHolderProfileFull.csv')
dfCardProfile.head(5)

In [ ]:
print(dfTrx.shape)
dfTrx=pd.merge(dfTrx, dfCardProfile, left_on='card_pan_id', 
               right_on='card_pan_id', how='left')


#print(dfTrx.head())
print("Fix clusterProfile")
print(dfTrx['clusterCardHolder'].value_counts())
dfTrx['clusterCardHolder'] = dfTrx['clusterCardHolder'].apply(lambda x: 'UNKNOWN' if pd.isnull(x) == True else x)
dfTrx.clusterCardHolder.value_counts()

# Get merchant profile

In [ ]:
import pandas as pd

dfMerchantProfile = pd.read_csv('../data/processed/'+'WLAcceptorProfile.csv')
dfMerchantProfile['term_mcc'] = dfMerchantProfile['term_mcc'].astype('str')
dfMerchantProfile.head(5)

In [ ]:
print(dfTrx.shape)
dfTrx=pd.merge(dfTrx, dfMerchantProfile, left_on=['acceptor_id','term_mcc','term_country'],
                right_on=['acceptor_id','term_mcc','term_country'], how='left')
print(dfTrx.shape)
print(dfTrx['clusterMerchant'].value_counts())
print("Fix merchantProfile")
dfTrx['clusterMerchant'] = dfTrx['clusterMerchant'].apply(lambda x: 'UNKNOWN' if pd.isnull(x) == True else x)
dfTrx.clusterMerchant.value_counts()

# Ecom or F2F

In [ ]:
dfTrx['ecom'] = np.where(dfTrx.card_entry_mode.isin([5,6,9]), 1,0)
dfTrx= dfTrx.drop(columns=['card_entry_mode'])

dfTrx.ecom.value_counts()

# One hot encoding

In [ ]:
dfTrx=pd.get_dummies(dfTrx,columns=['card_brand','country_group','mcc_group','trx_reversal','clusterCardHolder','clusterMerchant' ], dtype = int)
dfTrx.head()

# Previous transactions

In [ ]:

dfTrx['previous_trx']=0
dfTrx['wordV2']=dfTrx['term_country']+dfTrx['term_mcc']
sorted_df = dfTrx.sort_values(by=['card_pan_id','trx_date_time'])

sorted_df['card_pan_id1'] = sorted_df['card_pan_id'].shift(-1)
sorted_df['wordV2P'] = sorted_df['wordV2'].shift(-1)
dfTrx['wordV2P'] = sorted_df['wordV2P']
dfTrx['card_pan_id1']=sorted_df['card_pan_id1']

sorted_dfTemp=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id1'])]
dfTrx['previous_trx']=np.where(sorted_df.index.isin(sorted_dfTemp.index),1,dfTrx['previous_trx'])

dfTrx['distancePrevTrx']=1.1

f = open("../data/processed/wordV2.txt", "r")
text=f.read()

import gensim
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings 
warnings.filterwarnings(action='ignore')

data = []
for i in sent_tokenize(text):
    temp = [] 
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j) 
    data.append(temp)
model = Word2Vec(data,min_count=1,window=2,max_vocab_size=2000000)


def word_not_present(wordT,model):
    if (wordT in model.wv.key_to_index):
        return wordT
    else:
        #print(wordT)
        return 'BEL5411'

dfTrx['wordV2'] = dfTrx['wordV2'].apply(lambda x:word_not_present(x,model))
dfTrx['wordV2P'] = dfTrx['wordV2P'].apply(lambda x:word_not_present(x,model))

def calculDistance(input0,input1,input2,input3,model):
    if(input0!=input1):
        return 1.1
    else:
        return model.wv.similarity(input2,input3)
    

dfTrx['distancePrevTrx']= dfTrx.apply(lambda x: calculDistance(x.card_pan_id,x.card_pan_id1,x.wordV2P,x.wordV2,model), axis=1)


dfTrx= dfTrx.drop(columns=['wordV2','wordV2P','card_pan_id1'])
dfTrx.head()

# Trx_cnp (card not present) -> nothing to do

# Remove columns not yet managed

In [ ]:
dfTrx= dfTrx.drop(columns=['TRX_3D_SECURED','trx_accepted','trx_response_code',
                             'ecom_indicator','trx_authentication','pos_entry_mode','ch_present',
                            'acceptor_id'])
print(dfTrx.shape)
dfTrx.head()


# Save clean file

In [ ]:
dfTrx.to_csv('../data/cleaned/'+source+'export'+date+'.csv', index=False) 
print('done')

# Read saved file

In [ ]:
dfTrxSaved = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrxSaved.head(5)

In [ ]:
import dataimport as di
import pandas as pd 
   
dfTrx = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrx = di.remove_columns(dfTrx)
dfTrx = di.fill_missing_values(dfTrx)
dfTrx = di.category_grouping(dfTrx)
dfTrx = di.reversal_fix(dfTrx)
dfTrx = di.ecom(dfTrx)
dfTrx = di.join_card_holder_profile(dfTrx,'WLCardHolderProfileFull.csv')
dfTrx = di.join_merchant_profile(dfTrx,'WLAcceptorProfile.csv')
dfTrx = di.category_encoding(dfTrx)
dfTrx = di.amount_transformation(dfTrx)
dfTrx = di.previous_trx(dfTrx)
dfTrx = di.remove_column_not_yet_managed(dfTrx)
dfTrx.head(5)

In [ ]:
dfTrx = di.full_import_and_clean('../data/raw/'+source+'export'+date+'.csv','WLCardHolderProfileFull.csv','WLAcceptorProfile.csv')
dfTrx.to_csv('../data/cleaned/'+source+'export'+date+'.csv', index=False) 
dfTrxSaved = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrxSaved.head(5)

# Import file and clean a raw files

In [ ]:
dates =['20241118', '20241119','20241120','20241121','20241125']
for date in dates:
    print('process',date)
    dfTrx = di.full_import_and_clean('../data/raw/'+source+'export'+date+'.csv', 'WLCardHolderProfileFull.csv','WLAcceptorProfile.csv')
    dfTrx.to_csv('../data/cleaned/'+source+'export'+date+'.csv', index=False) 

print('done')
